In [ ]:
import torch
from torch import nn
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
import torchvision
import torchvision.transforms.v2 as v2
import os
import matplotlib.pyplot as plt
import numpy as np
import lightning as pl
from torchmetrics import Accuracy
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer
import wandb
import shutil
import numpy as np
import random
import gc
import sys
sys.path.append(os.path.abspath("src"))
from src.dataloader import iNat_dataset
from src.models import CNN_light
from src.config import *


: 

In [ ]:
os.environ['WANDB_API_KEY'] = "API-KEY"    # your API-KEY to be entered here
wandb.login(key=os.getenv("WANDB_API_KEY"))

In [ ]:
# Getting train, validation, and test dataloaders
dataset = iNat_dataset(data_dir=data_dir, augmentation = aug, batch_size=batch_size, NUM_WORKERS=NUM_WORKERS)
train_dataloader, val_dataloader, test_dataloader, n_classes = dataset.load_dataset()

In [ ]:
# Model training, evaluation
model = CNN_light(optim= optim, filters= filters, kernel = kernel, pool_kernel=pool_kernel, pool_stride=pool_stride, batchnorm=batchnorm, activation=activation, dropout=dropout, ffn_size=ffn_size, n_classses = n_classes, lr=lr)
logger= WandbLogger(project= 'CNN_scratch', name = 'best_model', log_model = False)
trainer = pl.Trainer(
                        devices=1,
                        accelerator="gpu",
                        precision="16-mixed",
                        gradient_clip_val=1.0,
                        max_epochs=epochs,
                        logger=logger,
                        profiler=None,
                        
                    )

trainer.fit(model, train_dataloader, val_dataloader)
trainer.test(model, dataloaders=test_dataloader)
pred_labels = trainer.predict(model, dataloaders=test_dataloader)

wandb.finish()

